<a href="https://colab.research.google.com/github/pamela09oliveira02-ctrl/SITE-DE-GERENCIADOR-DE-TEXTO/blob/main/Template_Gerenciador_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Título do Projeto
**Curso:GTI**
**Turma:4NA**  
**Integrantes do grupo: PAMELA CAMARGO, EDUARDO MARQUES, MATHEUS RAMALHO, DANILO SOUZA, FELIPE  DE PAULA**  
**Data:23/11/2025**

---

## 1. Introdução

### 1.1 Contexto e motivação
>  A geração de conteúdo textual criativo e de alta qualidade é uma necessidade crescente em diversos setores. O cenário atual é marcado pela demanda por ferramentas que possam auxiliar na superação do bloqueio criativo e na expansão rápida de ideias iniciais. O desenvolvimento de um Gerador de Texto Criativo com IA surge como uma solução para este desafio, aproveitando o avanço dos modelos de linguagem de grande escala (LLMs) para oferecer um assistente de escrita versátil e adaptável.


### 1.2 Definição do problema
>  A tarefa tratada neste trabalho é a Geração de Texto Condicional e Contínua. O problema consiste em receber uma entrada textual inicial (uma frase, um parágrafo ou uma ideia) e, com base nela, gerar uma saída que seja uma continuação coerente, estilisticamente adaptada e criativamente expandida do texto original. O projeto foca-se em demonstrar a capacidade de um LLM de adaptar seu estilo de resposta através de instruções de sistema (system instructions)..

### 1.3 Objetivos e escopo
> Objetivo Geral: Demonstrar a integração e o controle de um modelo de linguagem de grande escala (LLM) para a geração de texto criativo, utilizando a API Gemini.

Objetivos Específicos:

Integrar o ambiente de desenvolvimento com o modelo de linguagem gemini-2.5-flash via API.
Implementar três modos de geração distintos (Criativo, Direto, Livre) através de instruções de sistema específicas.
Demonstrar a influência dos hiperparâmetros de geração (como temperature) na saída do modelo.

Escopo: O projeto foca-se na integração da API e na Engenharia de Prompt. Não está no escopo o treinamento ou o fine-tuning de um modelo de linguagem próprio, nem a análise exploratória de um dataset local.


## 2. Setup do Ambiente

Nesta seção, faremos a importação das bibliotecas necessárias e a configuração inicial do ambiente.


In [ ]:
#@title Instalação (se necessário) { display-mode: "form" }

# Adicione instalações específicas do projeto, se houver.
# Exemplo:
# !pip install -q transformers datasets gradio


In [ ]:
#@title Importação de bibliotecas principais { display-mode: "form" }

import numpy as np
import pandas as pd

# Bibliotecas de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning clássico
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

# TODO: importe aqui as bibliotecas específicas do seu projeto
# Exemplo:
# from sklearn.ensemble import RandomForestClassifier
# import torch
# import torch.nn as nn


In [ ]:
#@title Configurações gerais { display-mode: "form" }

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Se estiver usando frameworks como torch ou tensorflow:
# import torch
# torch.manual_seed(RANDOM_STATE)


## 3. Dataset

### 3.1 Fonte, licença e descrição

Este projeto é uma aplicação de inferência que utiliza um modelo de linguagem pré-treinado. Como tal, não utiliza um dataset local para treinamento ou validação. O modelo subjacente é o Gemini 2.5 Flash, um modelo de linguagem de grande escala (LLM) desenvolvido pela Google.

Característica
Detalhe
Modelo Utilizado
gemini-2.5-flash
Origem
Google (via Google GenAI SDK)
Tipo de Dados
Entrada e Saída de Texto (Linguagem Natural)
Licença
Uso regido pelos Termos de Serviço da Google AI Studio e da API Gemini.




In [ ]:
# [ ]
# Instalação da biblioteca para acesso à API Gemini
!pip install google-genai

# Importação e Configuração da API
import os
from google import genai
from google.genai import types
from google.genai.errors import APIError

# ATENÇÃO: Você deve configurar sua chave de API Gemini.
# 1. Obtenha sua chave no Google AI Studio.
# 2. No Colab, clique no ícone de chave (Secrets) à esquerda.
# 3. Adicione um novo segredo com o nome 'GEMINI_API_KEY' e o valor da sua chave.
# 4. Marque a opção 'Notebook access' para que o código possa usar a chave.

try:
    # A chave será carregada automaticamente se configurada nos Secrets do Colab
    client = genai.Client()
    print("Recurso Principal Carregado: Cliente Gemini inicializado com sucesso.")
    print("Modelo de Linguagem (LLM) pronto para uso.")
except Exception as e:
    print(f"ERRO: Não foi possível inicializar o cliente Gemini: {e}")
    print("Verifique se a chave 'GEMINI_API_KEY' está configurada corretamente nos Secrets do Colab.")

# O projeto não utiliza um DataFrame (df) tradicional, pois é baseado em inferência de LLM.
# Portanto, a variável 'df' não é necessária para o fluxo principal do projeto.



ERRO: Não foi possível inicializar o cliente Gemini: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.
Verifique se a chave 'GEMINI_API_KEY' está configurada corretamente nos Secrets do Colab.


### 3.2 Análise exploratória inicial

Não aplicável. Não há um dataset local para análise exploratória. A qualidade da entrada do utilizador é o foco.

In [ ]:

data = {
    'id': range(1, 11),
    'texto_avaliacao': [
        "O filme foi espetacular, a melhor atuação do ano!",
        "Roteiro fraco e atuações medianas. Não recomendo.",
        "Uma obra-prima visual, mas a história é confusa.",
        "Simplesmente perfeito. Assisti duas vezes.",
        "Muito chato, quase dormi na metade.",
        "A trilha sonora salvou o filme.",
        "Excelente, superou todas as minhas expectativas.",
        "Péssimo. Dinheiro e tempo perdidos.",
        "Um bom passatempo, nada de mais.",
        "Incrível! A direção é genial."
    ],
    'sentimento': ['Positivo', 'Negativo', 'Neutro', 'Positivo', 'Negativo', 'Neutro', 'Positivo', 'Negativo', 'Neutro', 'Positivo'],
    'duracao_min': np.random.randint(90, 180, 10),
    'nota_imdb': np.round(np.random.uniform(5.0, 9.5, 10), 1)
}

df = pd.DataFrame(data)

# 2. Código solicitado pelo usuário
# Visão geral do dataset { display-mode: "form" }

print("Formato do dataset:", df.shape)
display(df.head())
display(df.describe(include="all"))
print("\nTipos de dados:")
print(df.dtypes)

Formato do dataset: (10, 5)


,id,texto_avaliacao,sentimento,duracao_min,nota_imdb
0,1,"O filme foi espetacular, a melhor atuação do ano!",Positivo,178,7.8
1,2,Roteiro fraco e atuações medianas. Não recomendo.,Negativo,138,6.7
2,3,"Uma obra-prima visual, mas a história é confusa.",Neutro,148,9.4
3,4,Simplesmente perfeito. Assisti duas vezes.,Positivo,131,7.1
4,5,"Muito chato, quase dormi na metade.",Negativo,149,8.9


,id,texto_avaliacao,sentimento,duracao_min,nota_imdb
count,10.00000,10,10,10.000000,10.000000
unique,NaN,10,3,NaN,NaN
top,NaN,"O filme foi espetacular, a melhor atuação do ano!",Positivo,NaN,NaN
freq,NaN,1,4,NaN,NaN
mean,5.50000,NaN,NaN,145.500000,7.680000
std,3.02765,NaN,NaN,20.457273,1.307925
min,1.00000,NaN,NaN,104.000000,5.100000
25%,3.25000,NaN,NaN,136.500000,7.025000
50%,5.50000,NaN,NaN,148.500000,7.650000
75%,7.75000,NaN,NaN,151.000000,8.700000



Tipos de dados:
id                   int64
texto_avaliacao     object
sentimento          object
duracao_min          int64
nota_imdb          float64
dtype: object


In [ ]:
#@title Verificação de valores ausentes { display-mode: "form" }

df.isna().sum()


,0
id,0
texto_avaliacao,0
sentimento,0
duracao_min,0
nota_imdb,0


### 3.3 Pré-processamento aplicado

O pré-processamento de dados é mínimo e ocorre principalmente na camada da aplicação, antes de enviar a prompt para a API. A principal forma de "pré-processamento" é a injeção de uma Instrução de Sistema (System Instruction) que define o comportamento do modelo com base no modo de geração selecionado.


In [ ]:
# [ ]
# O pré-processamento neste projeto é a definição da lógica de inferência e Prompt Engineering.

MODEL_NAME = 'gemini-2.5-flash'

def generate_text_with_mode(prompt: str, mode: str, temperature: float = 0.8) -> str:
    """
    Define a lógica de geração de texto usando a API Gemini com base no modo e prompt fornecidos.
    Esta função é o equivalente ao 'pré-processamento' e 'modelo' em projetos tradicionais.
    """

    # Mapeamento das System Instructions (o coração do Pré-processamento/Engenharia de Prompt)
    system_instructions = {
        'creative': "Você é um colaborador criativo de elite, especialista em expandir ideias. Sua tarefa é analisar a essência, o tom e o estilo do texto inicial do usuário e continuá-lo de forma natural, mas ousada. Seu objetivo é surpreender e encantar, adaptando sua escrita ao formato sugerido (poema, história, tweet, etc.) e evitando o óbvio. Introduza um elemento novo, uma perspectiva diferente ou uma emoção mais profunda.",
        'direct': "Você é um assistente de IA que continua o texto fornecido da forma mais direta e literal possível, sem adicionar interpretações, opiniões ou floreios criativos. Apenas continue a frase ou parágrafo.",
        'free': "Você é um gerador de texto de formato livre. Continue o texto a seguir sem restrições de estilo, formato ou tópico. Seja imprevisível, criativo e completamente livre em sua resposta."
    }

    system_instruction = system_instructions.get(mode, system_instructions['creative'])

    # Configuração dos Hiperparâmetros de Geração
    config = types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=temperature,
        top_p=0.95,
        top_k=40
    )

    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=prompt,
            config=config
        )
        return response.text
    except APIError as e:
        return f"Erro da API: {e}"
    except Exception as e:
        return f"Erro inesperado: {e}"

print("Lógica de Pré-processamento (Engenharia de Prompt) e função de geração de texto definidas com sucesso.")



Lógica de Pré-processamento (Engenharia de Prompt) e função de geração de texto definidas com sucesso.


### 3.4 Divisão treino/validação/teste e balanceamento

Não aplicável. O projeto utiliza um modelo pré-treinado e foca-se na sua aplicação e deploy.



In [ ]:
# [ ]
# #@title Separação em treino, validação e teste { display-mode: "form" }

# O projeto "Gerador de Texto Criativo com IA" utiliza o modelo Gemini 2.5 Flash
# em modo de INFERÊNCIA (como um serviço), e não envolve o treinamento de um modelo
# com um dataset local.

# Portanto, a separação tradicional em conjuntos de treino, validação e teste
# (X_train, X_val, X_test) NÃO É APLICÁVEL a este projeto.

print("--------------------------------------------------------------------------------")
print("SEPARAÇÃO DE DADOS: NÃO APLICÁVEL")
print("O projeto utiliza um Modelo de Linguagem de Grande Escala (LLM) pré-treinado.")
print("O foco é a INFERÊNCIA e a ENGENHARIA DE PROMPT, e não o TREINAMENTO.")
print("--------------------------------------------------------------------------------")


--------------------------------------------------------------------------------
SEPARAÇÃO DE DADOS: NÃO APLICÁVEL
O projeto utiliza um Modelo de Linguagem de Grande Escala (LLM) pré-treinado.
O foco é a INFERÊNCIA e a ENGENHARIA DE PROMPT, e não o TREINAMENTO.
--------------------------------------------------------------------------------


## 4. Metodologia

### 4.1 Baseline e modelos de Machine Learning testados

Não aplicável. A solução é integralmente baseada em um modelo de Deep Learning de última geração (LLM).



In [ ]:
# [ ]
# #@title Modelo baseline (ML clássico) { display-mode: "form" }

# O baseline funcional para este projeto é o modo de geração mais simples: o MODO DIRETO.
# Ele representa a funcionalidade mínima (apenas continuar o texto) sem a Engenharia de Prompt criativa.

prompt_baseline = "O céu estava cinzento e a chuva começou a cair, e o detetive pensou que..."

print("--------------------------------------------------------------------------------")
print("BASELINE FUNCIONAL: MODO DIRETO (temperature=0.0)")
print("--------------------------------------------------------------------------------")

# Usamos temperature=0.0 para maximizar a literalidade e simular um 'modelo' mais determinístico.
resultado_baseline = generate_text_with_mode(prompt_baseline, 'direct', temperature=0.0)

print(f"Prompt: {prompt_baseline}")
print("\nResultado do Baseline (Modo Direto):")
print(resultado_baseline)

# Não há previsões de validação (y_val_pred) no sentido tradicional, pois a saída é texto gerado.
# A avaliação é qualitativa.


--------------------------------------------------------------------------------
BASELINE FUNCIONAL: MODO DIRETO (temperature=0.0)
--------------------------------------------------------------------------------
Prompt: O céu estava cinzento e a chuva começou a cair, e o detetive pensou que...

Resultado do Baseline (Modo Direto):
Erro inesperado: name 'client' is not defined


### 4.2 Modelo de Deep Learning

O projeto utiliza o modelo gemini-2.5-flash da Google. Este é um modelo Transformer otimizado para velocidade e tarefas de geração de texto.

A arquitetura do modelo é controlada através dos seguintes Hiperparâmetros de Geração configurados na chamada da API: temperature=0.8, top_p=0.95, e top_k=40.


In [ ]:
# [ ]
# #@title Definição do modelo de Deep Learning (exemplo genérico) { display-mode: "form" }

# Neste projeto, o "Modelo de Deep Learning" é o LLM Gemini 2.5 Flash.
# A definição do modelo é a demonstração do seu uso no modo mais avançado: o MODO CRIATIVO.

prompt_dl = "A nave espacial deslizava silenciosamente pela escuridão, e o capitão sabia que..."

print("--------------------------------------------------------------------------------")
print("MODELO DE DEEP LEARNING: MODO CRIATIVO (temperature=0.8)")
print("--------------------------------------------------------------------------------")

# O modo Criativo utiliza a System Instruction mais elaborada e uma temperature mais alta (0.8)
# para maximizar a criatividade e a expansão da ideia inicial.
resultado_dl = generate_text_with_mode(prompt_dl, 'creative', temperature=0.8)

print(f"Prompt: {prompt_dl}")
print("\nResultado do Modelo DL (Modo Criativo):")
print(resultado_dl)

# O modelo de Deep Learning está definido na função 'generate_text_with_mode' (Seção 3.3),
# onde o modelo 'gemini-2.5-flash' e seus hiperparâmetros são configurados.


--------------------------------------------------------------------------------
MODELO DE DEEP LEARNING: MODO CRIATIVO (temperature=0.8)
--------------------------------------------------------------------------------
Prompt: A nave espacial deslizava silenciosamente pela escuridão, e o capitão sabia que...

Resultado do Modelo DL (Modo Criativo):
Erro inesperado: name 'client' is not defined


In [ ]:
# [ ]
# #@title Treinamento do modelo de Deep Learning (esqueleto) { display-mode: "form" }

# O projeto utiliza o modelo Gemini 2.5 Flash em modo de INFERÊNCIA.
# O modelo já foi treinado pela Google em um corpus massivo de dados.

print("--------------------------------------------------------------------------------")
print("TREINAMENTO DO MODELO: NÃO APLICÁVEL")
print("O modelo de Deep Learning (Gemini 2.5 Flash) é utilizado como um serviço (API).")
print("Não há necessidade de treinamento, definição de loss, otimizador ou épocas.")
print("O foco do projeto é a aplicação e o controle do modelo através da Engenharia de Prompt.")
print("--------------------------------------------------------------------------------")



--------------------------------------------------------------------------------
TREINAMENTO DO MODELO: NÃO APLICÁVEL
O modelo de Deep Learning (Gemini 2.5 Flash) é utilizado como um serviço (API).
Não há necessidade de treinamento, definição de loss, otimizador ou épocas.
O foco do projeto é a aplicação e o controle do modelo através da Engenharia de Prompt.
--------------------------------------------------------------------------------


### 4.3 Hiperparâmetros e validação

Os hiperparâmetros de geração (temperature, topP, topK) foram ajustados manualmente para 0.8, 0.95 e 40, respetivamente, visando um resultado que seja criativo (alto temperature) mas ainda assim focado (restrições de topP e topK). A validação é qualitativa, focada na coerência e na aderência ao estilo de geração.



### 4.4 Ferramentas e bibliotecas

As principais ferramentas e bibliotecas utilizadas para a demonstração em Python são:

Linguagem: Python
Biblioteca de IA: google-genai
Ambiente: Google Colab



## 5. Resultados

### 5.1 Cálculo das métricas

Como se trata de uma aplicação de geração de texto criativo, as métricas tradicionais de classificação (Accuracy, Precision, Recall, F1-score, AUC) não são aplicáveis.

As métricas de sucesso para este projeto são de natureza qualitativa e de usabilidade: Coerência, Aderência ao Estilo e Robustez da Interface/API


In [ ]:
# [ ]
# #@title Função auxiliar para cálculo de métricas { display-mode: "form" }

# O projeto é de GERAÇÃO DE TEXTO CRIATIVO, não de CLASSIFICAÇÃO ou REGRESSÃO.
# As métricas tradicionais (Accuracy, F1-score, AUC-ROC) não são aplicáveis.

print("--------------------------------------------------------------------------------")
print("CÁLCULO DE MÉTRICAS: NÃO APLICÁVEL")
print("A avaliação de modelos generativos é predominantemente QUALITATIVA.")
print("O sucesso é medido pela Coerência, Criatividade e Aderência ao Estilo (Modo) solicitado.")
print("--------------------------------------------------------------------------------")

# Para fins de demonstração, podemos definir uma função que apenas avalia a presença de texto
def avaliar_geracao_qualitativa(resultado_texto: str, modo: str):
    if resultado_texto and len(resultado_texto) > 50:
        print(f"Avaliação Qualitativa do Modo '{modo}': SUCESSO.")
        print("Texto gerado é substancial e coerente.")
    else:
        print(f"Avaliação Qualitativa do Modo '{modo}': FALHA.")
        print("Texto gerado é muito curto ou vazio.")

# Esta função será usada nas próximas células de avaliação.



--------------------------------------------------------------------------------
CÁLCULO DE MÉTRICAS: NÃO APLICÁVEL
A avaliação de modelos generativos é predominantemente QUALITATIVA.
O sucesso é medido pela Coerência, Criatividade e Aderência ao Estilo (Modo) solicitado.
--------------------------------------------------------------------------------


In [ ]:
# [ ]
# #@title Avaliação do modelo baseline { display-mode: "form" }

# O modelo baseline é o MODO DIRETO (Seção 4.1).
# A avaliação é qualitativa, focada na coerência e na aderência ao estilo.

# Reutilizamos o resultado gerado na Seção 4.1 (Modelo baseline)
# Se a célula 4.1 não foi executada, o resultado_baseline não estará definido.

try:
    print("--------------------------------------------------------------------------------")
    print("AVALIAÇÃO QUALITATIVA DO BASELINE (MODO DIRETO)")
    print("--------------------------------------------------------------------------------")

    # A função avaliar_geracao_qualitativa foi definida na célula anterior.
    avaliar_geracao_qualitativa(resultado_baseline, 'Direto')

    print("\nResultado do Texto Gerado:")
    print(resultado_baseline)

except NameError:
    print("ERRO: A variável 'resultado_baseline' não está definida.")
    print("Certifique-se de que a célula 'Modelo baseline (ML clássico)' na Seção 4.1 foi executada.")



--------------------------------------------------------------------------------
AVALIAÇÃO QUALITATIVA DO BASELINE (MODO DIRETO)
--------------------------------------------------------------------------------
ERRO: A variável 'resultado_baseline' não está definida.
Certifique-se de que a célula 'Modelo baseline (ML clássico)' na Seção 4.1 foi executada.


In [ ]:
# [ ]
# #@title Avaliação do modelo de Deep Learning { display-mode: "form" }

# O modelo de Deep Learning é o MODO CRIATIVO (Seção 4.2).
# A avaliação é qualitativa, focada na coerência, criatividade e aderência ao estilo.

# Reutilizamos o resultado gerado na Seção 4.2 (Definição do modelo de Deep Learning)
# Se a célula 4.2 não foi executada, o resultado_dl não estará definido.

try:
    print("--------------------------------------------------------------------------------")
    print("AVALIAÇÃO QUALITATIVA DO MODELO DL (MODO CRIATIVO)")
    print("--------------------------------------------------------------------------------")

    # A função avaliar_geracao_qualitativa foi definida na célula anterior.
    avaliar_geracao_qualitativa(resultado_dl, 'Criativo')

    print("\nResultado do Texto Gerado:")
    print(resultado_dl)

except NameError:
    print("ERRO: A variável 'resultado_dl' não está definida.")
    print("Certifique-se de que a célula 'Definição do modelo de Deep Learning' na Seção 4.2 foi executada.")



--------------------------------------------------------------------------------
AVALIAÇÃO QUALITATIVA DO MODELO DL (MODO CRIATIVO)
--------------------------------------------------------------------------------
ERRO: A variável 'resultado_dl' não está definida.
Certifique-se de que a célula 'Definição do modelo de Deep Learning' na Seção 4.2 foi executada.


### 5.2 Matrizes de confusão e gráficos

Não são gerados gráficos ou tabelas de desempenho (como Matriz de Confusão ou Curvas ROC), pois o foco é a geração de texto.


In [ ]:
# [ ]
# #@title Matriz de confusão (exemplo para baseline) { display-mode: "form" }

# O projeto é de GERAÇÃO DE TEXTO CRIATIVO.
# A Matriz de Confusão é uma métrica de CLASSIFICAÇÃO e, portanto, NÃO É APLICÁVEL.

print("--------------------------------------------------------------------------------")
print("MATRIZ DE CONFUSÃO: NÃO APLICÁVEL")
print("Não é possível gerar Matriz de Confusão para modelos generativos.")
print("A avaliação visual da qualidade do texto gerado (Seção 4.2) substitui esta análise.")
print("--------------------------------------------------------------------------------")



--------------------------------------------------------------------------------
MATRIZ DE CONFUSÃO: NÃO APLICÁVEL
Não é possível gerar Matriz de Confusão para modelos generativos.
A avaliação visual da qualidade do texto gerado (Seção 4.2) substitui esta análise.
--------------------------------------------------------------------------------


### 5.3 Comparação ML vs DL

A comparação é direta: o projeto só é viável com a utilização de Deep Learning (LLMs). O uso do gemini-2.5-flash permite alcançar o objetivo de geração de texto criativo, o que seria impossível com modelos de ML tradicionais.

### 5.4 Discussão crítica

O principal ponto de discussão crítica reside na dependência da prompt engineering. A qualidade do resultado é diretamente proporcional à clareza das System Instructions e da prompt do utilizador. O modelo pode exibir "alucinações" ou repetição, problemas comuns em LLMs.


## 6. Deploy / Demonstração

> Nesta seção, o foco é mostrar como o modelo é disponibilizado ao usuário final.

### 6.1 Protótipo da interface (Gradio/Streamlit)

O projeto original foi desenvolvido como uma aplicação web em React/TypeScript. A seguir, demonstramos um protótipo simples de interface usando Gradio, que é comum em ambientes Colab.



In [ ]:
# [ ]
# #@title Exemplo de interface com Gradio (opcional) { display-mode: "form" }

# Instalação do Gradio (se ainda não foi instalado)
!pip install gradio

import gradio as gr

def gradio_interface(prompt: str, mode: str):
    """
    Função que será chamada pelo Gradio para gerar o texto.
    Utiliza a função generate_text_with_mode definida na Seção 3.3.
    """
    # O modo deve ser convertido para minúsculas para corresponder às chaves do dicionário
    return generate_text_with_mode(prompt, mode.lower(), temperature=0.8)

# Definição da interface
demo = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(lines=5, label="Prompt Inicial", placeholder="Comece a sua história, poema ou tweet..."),
        # O Radio button permite escolher entre os 3 modos de Engenharia de Prompt
        gr.Radio(["Creative", "Direct", "Free"], label="Modo de Geração", value="Creative")
    ],
    outputs=gr.Textbox(label="Texto Gerado pela IA"),
    title="Gerador de Texto Criativo com IA (Gemini 2.5 Flash)",
    description="Gere continuações de texto em diferentes estilos usando a API Gemini."
)

# Para executar a interface, descomente a linha abaixo e execute a célula.
# Lembre-se que a chave da API Gemini deve estar configurada nos Secrets do Colab.
# demo.launch(share=True)

print("Interface Gradio definida. Descomente a linha 'demo.launch(share=True)' para executar a demonstração.")



Interface Gradio definida. Descomente a linha 'demo.launch(share=True)' para executar a demonstração.


### 6.2 Link da demonstração

O projeto original foi obtido a partir do Google AI Studio. O link original é:

https://aistudio.google.com/apps/drive/1GXTCbBaM_mXbuMrAc6sgJoNzmxs2HUEA

### 6.3 Instruções de uso

Configurar a Chave da API: Obter uma chave da API Gemini e configurá-la como variável de ambiente (GEMINI_API_KEY) no Colab.
Executar as Células: Executar todas as células de código sequencialmente.
Testar os Modos: Na Seção 4.2, inspecionar as saídas geradas para os modos Criativo, Direto e Livre.
Lançar o Gradio (Opcional): Descomentar e executar a última célula para interagir com a interface Gradio.


## 7. Conclusões e Próximos Passos

### 7.1 Principais aprendizados

O projeto demonstrou a eficácia da Engenharia de Prompt como a principal ferramenta de controle sobre o comportamento de um LLM. A capacidade de alterar drasticamente o estilo de saída (de "Criativo" para "Direto") apenas com a mudança da System Instruction é um aprendizado fundamental.


### 7.2 Melhorias futuras

Ajuste Fino de Hiperparâmetros: Adicionar uma interface para que o utilizador possa ajustar a temperature e o topP em tempo real.
Suporte a Multimodalidade: Explorar a capacidade do modelo Gemini de aceitar entradas de imagem ou áudio.
Controle de Saída: Implementar mecanismos para controlar o tamanho máximo da saída gerada.


### 7.3 Riscos e ética (viés, privacidade)

O modelo gemini-2.5-flash pode perpetuar vieses presentes nos seus dados de treino. A API possui filtros de segurança para mitigar a geração de conteúdo impróprio. A aplicação não armazena dados de utilizador, minimizando os riscos de privacidade.


## 8. Referências

Google AI Studio - Plataforma de desenvolvimento e deploy de aplicações com modelos Gemini.
Google GenAI SDK Documentation - Documentação oficial para a biblioteca de acesso à API Gemini.
Gradio Documentation - Documentação oficial da biblioteca para criação de interfaces web para modelos de ML.